In [28]:
import tensorflow as tf
import numpy as np
from examples.iris import IrisDataset
from sklearn.metrics import accuracy_score


rnd = np.random.RandomState(seed=13)
n_units = 100
n_in = 4
n_out = 3

csv_path = "./examples/"
dataset = IrisDataset(csv_path=csv_path, seed=12)

x = tf.placeholder(dtype=tf.float32, shape=(None, n_in), name="ExternalInput")
W_init = rnd.normal(size=(n_in, n_units), loc=0, scale=.01)
b_init = np.zeros(shape=(n_units,))
W = tf.Variable(initial_value=W_init, name='W', dtype=tf.float32)
b = tf.Variable(initial_value=b_init, name='b', dtype=tf.float32)
a = tf.matmul(x, W) + b
h = tf.tanh(a)
W_out_init = rnd.normal(size=(n_units, n_out), loc=0, scale=.01)
W_out = tf.Variable(initial_value=W_out_init, name='W_out', dtype=tf.float32)

b_out_init = np.zeros(shape=(n_out,))
b_out = tf.Variable(initial_value=b_out_init, name='b_out', dtype=tf.float32)
y = tf.matmul(h, W_out) + b_out

In [29]:
# task
t = tf.placeholder(tf.float32, shape=[None, n_out], name="labels")  # labels
loss = -tf.reduce_sum(t * tf.log(y), reduction_indices=[1])

In [30]:
# training step
alpha = 0.001
gW_out, gb_out = tf.gradients(loss, [W_out, b_out])
gh = tf.gradients(loss, h)[0]
h_1 = h - alpha * gh

# a_star = a  # TODO
cutoff = 0.99


def arctanh(x):
    return (tf.log(1 + x) - tf.log(1 - x)) / 2

a_star = tf.where(tf.logical_and(h_1 > h, h > cutoff), a,
                  tf.where(tf.logical_and(h_1 < h, h < -cutoff), a, arctanh(h_1)))

d = a_star - a

ga_W_in, ga_b_in = tf.gradients(tf.reduce_mean((a - a_star) ** 2, axis=0), [W, b])
# ga_W_in, ga_b_in = tf.gradients(loss, [W, b])

grad_var_pairs = ((gW_out, W_out), (gb_out, b_out), (ga_b_in, b), (ga_W_in, W))
grads = [tf.reshape(g[0], [-1]) for g in grad_var_pairs]
gradient = tf.concat(grads, 0)

clipped_grads, _ = tf.clip_by_global_norm(grads, clip_norm=1)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=alpha)
train_op = optimizer.apply_gradients(grad_var_pairs)

In [ ]:
import sys
sys.stdout = open('/dev/stdout', 'w')

batch_size = 20

sess = tf.InteractiveSession()
sess.run(tf.local_variables_initializer())
sess.run(tf.global_variables_initializer())

validation_batch = dataset.get_validation()
test_batch = dataset.get_test()

print("Training...")
for i in range(300):
    batch = dataset.get_batch(batch_size)
    feed_dict = {x: batch["input"], t: batch["output"]}
    _, grad_norm = sess.run((train_op, tf.reduce_sum(tf.reduce_sum(gradient**2))), feed_dict=feed_dict)
    if i % 100 == 0:
        y_np = sess.run(y, feed_dict={x: validation_batch["input"], t: validation_batch["output"]})
        acc = accuracy_score(y_true=np.argmax(validation_batch["output"], axis=1), y_pred=np.argmax(y_np, axis=1))
        # print("Val accuracy score: {:.2f}".format(acc))

        print('Iteration {} -> val error:{:.2f}, g_norm: {:.2f}'.format(i, acc, grad_norm))
        sys.stdout.flush()

In [32]:
y_np = sess.run(y, feed_dict={x: test_batch["input"]})

score = accuracy_score(y_true=np.argmax(dataset.get_test()["output"], axis=1), y_pred=np.argmax(y_np, axis=1))
sess.close()
print("Test accuracy score: {:.2f}".format(score))


0.26666666666666666